# Garmin Activities Data Exploration

This notebook explores the Activities module from Garmin Connect API.

## Available Activities Methods:
- `get_activities(start, limit)` - Fetch activities with pagination
- `get_activities_fordate(date)` - Activities for specific date
- `get_last_activity()` - Most recent activity
- `get_activity(activity_id)` - Activity summary
- `get_activity_details(activity_id)` - Comprehensive activity data including splits
- `get_activity_gear(activity_id)` - Gear used in activity

## Goal:
Extract activities data and create dbt models that can be joined with gear data!

In [1]:
# Cell 1: Imports and Login
from garminconnect import Garmin
from datetime import date, timedelta
import json
import pandas as pd
import os
from getpass import getpass

# Login using stored tokens
tokenstore = os.path.expanduser("~/.garminconnect")

try:
    print(f"Attempting to login using stored tokens from: {tokenstore}")
    api = Garmin()
    api.login(tokenstore)
    print("✅ Successfully logged in using stored tokens!")
except Exception:
    print("No valid tokens found. Requesting fresh login credentials.")
    email = input("Email address: ").strip()
    password = getpass("Password: ")
    
    api = Garmin(email=email, password=password)
    api.login()
    api.garth.dump(tokenstore)
    print(f"✅ Login successful! Tokens saved to: {tokenstore}")

today = date.today()
print(f"Connected! Today's date: {today}")

Attempting to login using stored tokens from: C:\Users\Svitlana/.garminconnect
✅ Successfully logged in using stored tokens!
Connected! Today's date: 2025-11-10


In [3]:
# Cell 2: Get Recent Activities (Basic List)
print("🏃 RECENT ACTIVITIES")
print("="*60)

# Get last 10 activities
activities = api.get_activities(0, 10)

print(f"\nFound {len(activities)} activities:\n")

for i, activity in enumerate(activities, 1):
    activity_id = activity.get('activityId')
    name = activity.get('activityName', 'Unnamed')
    activity_type = activity.get('activityType', {}).get('typeKey', 'unknown')
    distance = activity.get('distance', 0) / 1000  # meters to km
    duration = activity.get('duration', 0) / 60  # seconds to minutes
    start_time = activity.get('startTimeLocal', 'N/A')
    
    print(f"{i}. [{activity_id}] {name}")
    print(f"   Type: {activity_type}")
    print(f"   Distance: {distance:.2f} km")
    print(f"   Duration: {duration:.1f} minutes")
    print(f"   Start: {start_time}")
    print()

🏃 RECENT ACTIVITIES

Found 10 activities:

1. [20944376123] Copenhagen Running
   Type: running
   Distance: 7.61 km
   Duration: 40.7 minutes
   Start: 2025-11-10 06:42:22

2. [20925094030] Copenhagen Running
   Type: running
   Distance: 5.01 km
   Duration: 28.8 minutes
   Start: 2025-11-08 07:52:30

3. [20906728222] Strength
   Type: strength_training
   Distance: 0.00 km
   Duration: 42.0 minutes
   Start: 2025-11-06 07:29:08

4. [20886162853] Strength
   Type: strength_training
   Distance: 0.00 km
   Duration: 51.5 minutes
   Start: 2025-11-04 07:37:04

5. [20878081860] Strength
   Type: strength_training
   Distance: 0.00 km
   Duration: 52.4 minutes
   Start: 2025-11-03 07:56:21

6. [20867221680] Copenhagen Cycling
   Type: cycling
   Distance: 24.84 km
   Duration: 54.2 minutes
   Start: 2025-11-02 10:42:45

7. [20822731404] Cardio
   Type: indoor_cardio
   Distance: 0.00 km
   Duration: 36.0 minutes
   Start: 2025-10-28 16:20:09

8. [20818494339] Cardio
   Type: indoor_cardi

In [4]:
# Cell 3: View Raw Data Structure
print("📋 RAW ACTIVITY DATA STRUCTURE")
print("="*60)

# Get the first activity for inspection
if activities:
    sample_activity = activities[0]
    print(f"\nActivity ID: {sample_activity.get('activityId')}")
    print(f"Activity Name: {sample_activity.get('activityName')}")
    print(f"\nAvailable fields ({len(sample_activity)} total):\n")
    
    # Show all keys
    for key in sorted(sample_activity.keys()):
        value = sample_activity[key]
        print(f"  {key}: {type(value).__name__}")
    
    print("\n" + "="*60)
    print("Full JSON:")
    print("="*60)
    print(json.dumps(sample_activity, indent=2, default=str))

📋 RAW ACTIVITY DATA STRUCTURE

Activity ID: 20944376123
Activity Name: Copenhagen Running

Available fields (81 total):

  activityId: int
  activityName: str
  activityType: dict
  aerobicTrainingEffect: float
  aerobicTrainingEffectMessage: str
  anaerobicTrainingEffect: float
  anaerobicTrainingEffectMessage: str
  atpActivity: bool
  autoCalcCalories: bool
  averageHR: float
  averageRunningCadenceInStepsPerMinute: float
  averageSpeed: float
  avgStrideLength: float
  beginTimestamp: int
  bmrCalories: float
  calories: float
  decoDive: bool
  deviceId: int
  distance: float
  duration: float
  elapsedDuration: float
  elevationCorrected: bool
  elevationGain: float
  elevationLoss: float
  endLatitude: float
  endLongitude: float
  endTimeGMT: str
  eventType: dict
  fastestSplit_1000: float
  fastestSplit_1609: float
  fastestSplit_5000: float
  favorite: bool
  hasHeatMap: bool
  hasImages: bool
  hasPolyline: bool
  hasSplits: bool
  hasVideo: bool
  hrTimeInZone_1: float
  h

In [5]:
# Cell 4: Get Detailed Activity Data
print("🔍 DETAILED ACTIVITY DATA")
print("="*60)

if activities:
    # Get details for the first activity
    activity_id = activities[0].get('activityId')
    print(f"\nFetching details for activity: {activity_id}\n")
    
    # Get detailed activity data
    details = api.get_activity(activity_id)
    
    print(f"Activity: {details.get('activityName')}")
    print(f"Type: {details.get('activityType', {}).get('typeKey')}")
    print(f"\nKey Metrics:")
    print(f"  Distance: {details.get('distance', 0)/1000:.2f} km")
    print(f"  Duration: {details.get('duration', 0)/60:.1f} minutes")
    print(f"  Avg Speed: {details.get('averageSpeed', 0)*3.6:.2f} km/h")
    print(f"  Avg HR: {details.get('averageHR', 'N/A')} bpm")
    print(f"  Max HR: {details.get('maxHR', 'N/A')} bpm")
    print(f"  Calories: {details.get('calories', 'N/A')}")
    print(f"  Elevation Gain: {details.get('elevationGain', 'N/A')} m")
    
    print(f"\n\nAvailable fields in detailed data: {len(details)}")
    print("\nField names:")
    for key in sorted(details.keys()):
        print(f"  - {key}")

🔍 DETAILED ACTIVITY DATA

Fetching details for activity: 20944376123

Activity: Copenhagen Running
Type: None

Key Metrics:
  Distance: 0.00 km
  Duration: 0.0 minutes
  Avg Speed: 0.00 km/h
  Avg HR: N/A bpm
  Max HR: N/A bpm
  Calories: N/A
  Elevation Gain: N/A m


Available fields in detailed data: 13

Field names:
  - accessControlRuleDTO
  - activityId
  - activityName
  - activityTypeDTO
  - activityUUID
  - eventTypeDTO
  - isMultiSportParent
  - locationName
  - metadataDTO
  - splitSummaries
  - summaryDTO
  - timeZoneUnitDTO
  - userProfileId


In [6]:
# Cell 5: Check Activity Gear Connection
print("👟 ACTIVITY GEAR CONNECTION")
print("="*60)

if activities:
    activity_id = activities[0].get('activityId')
    
    try:
        # Get gear used in this activity
        gear = api.get_activity_gear(activity_id)
        
        print(f"\nGear used in activity {activity_id}:\n")
        print(json.dumps(gear, indent=2, default=str))
        
    except Exception as e:
        print(f"Could not fetch gear info: {e}")
    
    # Also check if gear info is in the main activity data
    print("\n" + "="*60)
    print("Checking for gear in main activity data:")
    print("="*60)
    
    if 'gearUuid' in activities[0]:
        print(f"\nGear UUID: {activities[0].get('gearUuid')}")
    else:
        print("\nNo direct gear UUID in activity summary")
    
    # Show all gear-related fields
    gear_fields = [k for k in activities[0].keys() if 'gear' in k.lower()]
    if gear_fields:
        print(f"\nGear-related fields found:")
        for field in gear_fields:
            print(f"  {field}: {activities[0].get(field)}")

👟 ACTIVITY GEAR CONNECTION

Gear used in activity 20944376123:

[
  {
    "gearPk": 43608073,
    "uuid": "74d093ce9ed34e409d2df054e53d91f3",
    "userProfilePk": 89635332,
    "gearMakeName": "Other",
    "gearModelName": "Unknown Shoes",
    "gearTypeName": "Shoes",
    "gearStatusName": "active",
    "displayName": null,
    "customMakeModel": "On Cloud Eclipse",
    "imageNameLarge": null,
    "imageNameMedium": null,
    "imageNameSmall": null,
    "dateBegin": "2025-05-18T00:00:00.0",
    "dateEnd": null,
    "maximumMeters": 900000.0,
    "notified": false,
    "createDate": "2025-05-18T15:40:55.0",
    "updateDate": "2025-05-18T15:40:55.0"
  }
]

Checking for gear in main activity data:

No direct gear UUID in activity summary


In [ ]:
# Cell 6: Convert to DataFrame for Analysis
print("📊 ACTIVITIES DATAFRAME")
print("="*60)

# Convert activities to DataFrame
df = pd.DataFrame(activities)

print(f"\nShape: {df.shape[0]} activities, {df.shape[1]} columns")
print(f"\nColumn names:")
print(df.columns.tolist())

print(f"\n\nFirst few activities:")
display(df[['activityId', 'activityName', 'startTimeLocal', 'distance', 'duration', 'calories']].head())

print(f"\n\nData types:")
print(df.dtypes)

In [ ]:
# Cell 7: Get More Activities (Last 100)
print("📥 FETCHING LARGER DATASET")
print("="*60)

# Get last 100 activities
print("\nFetching last 100 activities...")
all_activities = api.get_activities(0, 100)

print(f"✅ Retrieved {len(all_activities)} activities")

# Convert to DataFrame
df_all = pd.DataFrame(all_activities)

# Show summary statistics
print("\n" + "="*60)
print("DATASET SUMMARY")
print("="*60)

print(f"\nTotal activities: {len(df_all)}")
print(f"Date range: {df_all['startTimeLocal'].min()} to {df_all['startTimeLocal'].max()}")

if 'activityType' in df_all.columns:
    # Extract activity type key
    df_all['activity_type'] = df_all['activityType'].apply(lambda x: x.get('typeKey') if isinstance(x, dict) else 'unknown')
    print(f"\nActivity types:")
    print(df_all['activity_type'].value_counts())

print(f"\nTotal distance: {df_all['distance'].sum()/1000:.2f} km")
print(f"Total duration: {df_all['duration'].sum()/3600:.1f} hours")
print(f"Total calories: {df_all['calories'].sum():,.0f}")

In [ ]:
# Cell 8: Save to Database
import sqlite3

print("💾 SAVING TO DATABASE")
print("="*60)

# Database path (adjust if needed)
db_path = '../garmin.db'

# Connect to database
conn = sqlite3.connect(db_path)

# Save activities to bronze table
df_all.to_sql('bronze_activities', conn, if_exists='replace', index=False)

print(f"✅ Saved {len(df_all)} activities to 'bronze_activities' table")

# Verify
result = pd.read_sql("SELECT COUNT(*) as count FROM bronze_activities", conn)
print(f"\nVerification: {result['count'][0]} records in database")

conn.close()
print("\n✨ Done!")

In [8]:
# Cell 5: Check Activity Gear Connection - Top 10 Activities
import time

print("👟 ACTIVITY GEAR CONNECTION - Top 10 Activities")
print("="*60)

if activities:
    gear_summary = []
    
    # Check first 10 activities
    for i in range(min(10, len(activities))):
        activity = activities[i]
        activity_id = activity.get('activityId')
        activity_name = activity.get('activityName', 'Unnamed')
        
        print(f"\n{i+1}. Checking activity: {activity_name[:40]} (ID: {activity_id})")
        
        # Try to get gear from API
        gear_from_api = None
        try:
            gear_from_api = api.get_activity_gear(activity_id)
            print(f"   ✅ Gear from API: {gear_from_api}")
        except Exception as e:
            print(f"   ⚠️ Could not fetch gear from API: {e}")
        
        # Check if gear UUID is in the activity data
        gear_uuid = activity.get('gearUuid')
        print(f"   Gear UUID in data: {gear_uuid}")
        
        # Collect all gear-related fields
        gear_fields_data = {k: activity.get(k) for k in activity.keys() if 'gear' in k.lower()}
        
        # Add to summary
        gear_summary.append({
            'activityId': activity_id,
            'activityName': activity_name,
            'gearUuid': gear_uuid,
            'gear_from_api': gear_from_api,
            'all_gear_fields': gear_fields_data
        })
        
        time.sleep(0.3)  # Be nice to Garmin's servers
    
    # Display summary as DataFrame
    print("\n" + "="*60)
    print("📊 GEAR SUMMARY - First 10 Activities (All Columns)")
    print("="*60)
    
    df_gear_summary = pd.DataFrame(gear_summary)
    
    # Set display options to show all columns
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 100)
    
    print(df_gear_summary)
    
    # Show statistics
    activities_with_gear = df_gear_summary['gearUuid'].notna().sum()
    print(f"\n📈 {activities_with_gear} out of 10 activities have gear assigned")
    
    # Show all unique gear-related field names
    print("\n" + "="*60)
    print("🔍 All gear-related fields found in activities:")
    print("="*60)
    
    all_gear_field_names = set()
    for fields_dict in gear_summary:
        all_gear_field_names.update(fields_dict['all_gear_fields'].keys())
    
    if all_gear_field_names:
        for field in sorted(all_gear_field_names):
            print(f"  - {field}")
    else:
        print("  No gear-related fields found")

else:
    print("No activities available")

👟 ACTIVITY GEAR CONNECTION - Top 10 Activities

1. Checking activity: Copenhagen Running (ID: 20944376123)
   ✅ Gear from API: [{'gearPk': 43608073, 'uuid': '74d093ce9ed34e409d2df054e53d91f3', 'userProfilePk': 89635332, 'gearMakeName': 'Other', 'gearModelName': 'Unknown Shoes', 'gearTypeName': 'Shoes', 'gearStatusName': 'active', 'displayName': None, 'customMakeModel': 'On Cloud Eclipse', 'imageNameLarge': None, 'imageNameMedium': None, 'imageNameSmall': None, 'dateBegin': '2025-05-18T00:00:00.0', 'dateEnd': None, 'maximumMeters': 900000.0, 'notified': False, 'createDate': '2025-05-18T15:40:55.0', 'updateDate': '2025-05-18T15:40:55.0'}]
   Gear UUID in data: None

2. Checking activity: Copenhagen Running (ID: 20925094030)
   ✅ Gear from API: [{'gearPk': 43608073, 'uuid': '74d093ce9ed34e409d2df054e53d91f3', 'userProfilePk': 89635332, 'gearMakeName': 'Other', 'gearModelName': 'Unknown Shoes', 'gearTypeName': 'Shoes', 'gearStatusName': 'active', 'displayName': None, 'customMakeModel': 'On

In [9]:
import pandas as pd
import time

print("👟 Testing api.get_activity_gear() - First 10 Activities")
print("="*60)

# Get first 10 activities
activities = api.get_activities(0, 10)

gear_results = []

for i, activity in enumerate(activities):
    activity_id = activity.get('activityId')
    activity_name = activity.get('activityName', 'Unnamed')
    
    print(f"\n{i+1}. {activity_name[:50]} (ID: {activity_id})")
    
    try:
        gear = api.get_activity_gear(activity_id)
        print(f"   ✅ Got gear data: {type(gear)}")
        
        # If it's a dict, add it to results
        if isinstance(gear, dict):
            gear['activityId'] = activity_id
            gear['activityName'] = activity_name
            gear_results.append(gear)
        # If it's a list, add each item
        elif isinstance(gear, list):
            for g in gear:
                g['activityId'] = activity_id
                g['activityName'] = activity_name
                gear_results.append(g)
        else:
            print(f"   ⚠️  Unexpected type: {gear}")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")
    
    time.sleep(0.5)  # Be nice to API

# Show results
print("\n" + "="*60)
print("📊 ALL GEAR DATA")
print("="*60)

if gear_results:
    df = pd.DataFrame(gear_results)
    
    # Show all columns
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    
    print(df)
    print(f"\n✅ Found gear data for {len(df)} activities")
    print(f"📋 Columns: {list(df.columns)}")
else:
    print("❌ No gear data found")

👟 Testing api.get_activity_gear() - First 10 Activities

1. Copenhagen Running (ID: 20944376123)
   ✅ Got gear data: <class 'list'>

2. Copenhagen Running (ID: 20925094030)
   ✅ Got gear data: <class 'list'>

3. Strength (ID: 20906728222)
   ✅ Got gear data: <class 'list'>

4. Strength (ID: 20886162853)
   ✅ Got gear data: <class 'list'>

5. Strength (ID: 20878081860)
   ✅ Got gear data: <class 'list'>

6. Copenhagen Cycling (ID: 20867221680)
   ✅ Got gear data: <class 'list'>

7. Cardio (ID: 20822731404)
   ✅ Got gear data: <class 'list'>

8. Cardio (ID: 20818494339)
   ✅ Got gear data: <class 'list'>

9. Copenhagen Running (ID: 20813575207)
   ✅ Got gear data: <class 'list'>

10. Copenhagen Running (ID: 20790858054)
   ✅ Got gear data: <class 'list'>

📊 ALL GEAR DATA
     gearPk                              uuid  userProfilePk gearMakeName  \
0  43608073  74d093ce9ed34e409d2df054e53d91f3       89635332        Other   
1  43608073  74d093ce9ed34e409d2df054e53d91f3       89635332      

In [11]:
# ... (keep all the code above)

# Show results
print("\n" + "="*60)
print("📊 ALL GEAR DATA")
print("="*60)

if gear_results:
    df = pd.DataFrame(gear_results)
    
    # Show all columns
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    
    print(df)
    print(f"\n✅ Found gear data for {len(df)} activities")
    print(f"📋 Columns: {list(df.columns)}")
    

    # Save in the same folder as your notebook
    df.to_excel('activity_gear_test.xlsx', index=False)
    print("💾 Saved to: activity_gear_test.xlsx")
    
else:
    print("❌ No gear data found")


📊 ALL GEAR DATA
     gearPk                              uuid  userProfilePk gearMakeName  \
0  43608073  74d093ce9ed34e409d2df054e53d91f3       89635332        Other   
1  43608073  74d093ce9ed34e409d2df054e53d91f3       89635332        Other   
2  45528920  ad3545999f2f46558c214d909f4ec0fb       89635332        Other   
3  43608073  74d093ce9ed34e409d2df054e53d91f3       89635332        Other   

   gearModelName gearTypeName gearStatusName displayName   customMakeModel  \
0  Unknown Shoes        Shoes         active        None  On Cloud Eclipse   
1  Unknown Shoes        Shoes         active        None  On Cloud Eclipse   
2  Unknown Shoes        Shoes         active        None   Asics Megablast   
3  Unknown Shoes        Shoes         active        None  On Cloud Eclipse   

  imageNameLarge imageNameMedium imageNameSmall              dateBegin  \
0           None            None           None  2025-05-18T00:00:00.0   
1           None            None           None  2025-05-1